In [ ]:
!pip3 install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
